In [2]:
import pandas as pd
themes_sets = pd.read_csv('prices_sets_age_themes.csv')
themes_sets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24505 entries, 0 to 24504
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Packaging          14936 non-null  object 
 1   Num_Instructions   14936 non-null  float64
 2   Availability       14936 non-null  object 
 3   Owned              14771 non-null  float64
 4   Rating             14936 non-null  float64
 5   Total_Quantity     12276 non-null  float64
 6   Current_Price      5442 non-null   float64
 7   set_id             24505 non-null  object 
 8   name               24505 non-null  object 
 9   year               24505 non-null  float64
 10  agerange_min       6787 non-null   float64
 11  bricksetURL        18457 non-null  object 
 12  thumbnailURL       17451 non-null  object 
 13  num_pieces         23126 non-null  float64
 14  set_theme          18958 non-null  object 
 15  set_theme_group    18937 non-null  object 
 16  set_subtheme       153

In [3]:
themes_sets['set_theme'].nunique()

160

In [4]:
themes_sets['set_theme_group'].nunique()

16

In [5]:
themes_sets['r_main_theme'].nunique() # it would be great if we could categorize these Rebrickable main themes into the set theme groups and set themes above

137

In [6]:
themes_sets[themes_sets['set_theme']==themes_sets['r_main_theme']].info() #in about half of the rows, the R. main theme is the same as the set theme - I can use this

<class 'pandas.core.frame.DataFrame'>
Index: 12522 entries, 2 to 23459
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Packaging          10339 non-null  object 
 1   Num_Instructions   10339 non-null  float64
 2   Availability       10339 non-null  object 
 3   Owned              10239 non-null  float64
 4   Rating             10339 non-null  float64
 5   Total_Quantity     9217 non-null   float64
 6   Current_Price      4229 non-null   float64
 7   set_id             12522 non-null  object 
 8   name               12522 non-null  object 
 9   year               12522 non-null  float64
 10  agerange_min       4594 non-null   float64
 11  bricksetURL        12174 non-null  object 
 12  thumbnailURL       11698 non-null  object 
 13  num_pieces         12522 non-null  float64
 14  set_theme          12522 non-null  object 
 15  set_theme_group    12510 non-null  object 
 16  set_subtheme       10170 no

In [7]:
themes = themes_sets[['set_theme','set_theme_group','r_main_theme']]
themes.dropna(subset=['r_main_theme'],inplace=True)
themes.drop_duplicates(inplace=True)
themes.info()
#this table could be used to attribute a set theme group to a Rebrickable main theme in cases where the set theme is the same as the R. main theme

<class 'pandas.core.frame.DataFrame'>
Index: 434 entries, 2 to 23403
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   set_theme        338 non-null    object
 1   set_theme_group  333 non-null    object
 2   r_main_theme     434 non-null    object
dtypes: object(3)
memory usage: 13.6+ KB


C:\Users\A\AppData\Local\Temp\ipykernel_13592\2306565051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  themes.dropna(subset=['r_main_theme'],inplace=True)
C:\Users\A\AppData\Local\Temp\ipykernel_13592\2306565051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  themes.drop_duplicates(inplace=True)


In [8]:
themes.groupby('set_theme').filter(lambda x: x['set_theme_group'].nunique()>1) #there is always max 1 set theme group per set theme which is what we need to always be able to attribute a set theme group to a set theme

,set_theme,set_theme_group,r_main_theme


In [9]:
h = themes[themes['set_theme']==themes['r_main_theme']]
#now I have a data frame with only those rows where the theme and R. main theme are the same
h.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, 2 to 19607
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   set_theme        95 non-null     object
 1   set_theme_group  94 non-null     object
 2   r_main_theme     95 non-null     object
dtypes: object(3)
memory usage: 3.0+ KB


In [12]:
sets = pd.merge(themes_sets,h,how='left',on='r_main_theme') #mergin with the data frame containing information on sets, prices and themes
sets['set_theme_x'] = sets[['set_theme_x','set_theme_y']].bfill(axis=1).iloc[:, 0]
sets['set_theme_group_x'] = sets[['set_theme_group_x','set_theme_group_y']].bfill(axis=1).iloc[:, 0]
sets.drop(columns=['set_theme_y','set_theme_group_y'],inplace=True)
sets.rename(columns={'set_theme_x':'set_theme','set_theme_group_x':'set_theme_group'},inplace=True)
sets[['set_theme','set_theme_group']].info() # 23058 rows with a set theme group compared with the original 18937 rows - that's 4000 additional rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24505 entries, 0 to 24504
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   set_theme        23095 non-null  object
 1   set_theme_group  23058 non-null  object
dtypes: object(2)
memory usage: 383.0+ KB


In [11]:
sets.to_csv('prices_sets_age_themesE.csv',index=False)